In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm

from mpmath import besseljzero
from scipy.special import gamma
from scipy.special import jv, iv, ive
from scipy.optimize import minimize
from scipy.optimize import differential_evolution
from scipy.interpolate import interp1d

from scipy.stats import pearsonr
from sklearn.metrics import r2_score

import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
def simulate_HSDM_4D(a, mu, eta, ndt, sigma=1, dt=0.001):
    x = np.zeros(mu.shape)
    
    rt = 0
    
    mu_t = np.random.normal(mu, eta)
    while np.linalg.norm(x, 2) < a(rt):
        x += mu_t*dt + sigma*np.sqrt(dt)*np.random.normal(0, 1, mu.shape)
        rt += dt
    
    theta1 = np.arctan2(np.sqrt(x[3]**2 + x[2]**2 + x[1]**2), x[0])
    theta2 = np.arctan2(np.sqrt(x[3]**2 + x[2]**2), x[1])
    theta3 = np.arctan2(x[3], x[2])
    
    
    return rt+ndt, (theta1, theta2, theta3)

In [3]:
def k(a, da, t, q, sigma=2):
    return 0.5 * (q - 0.5*sigma - da(t))

def psi(a, da, t, z, tau, q, sigma=2):
    kk = k(a, da, t, q, sigma)
    
    if 2*np.sqrt(a(t)*z)/(sigma*(t-tau))<=700:
        term1 = 1./(sigma*(t - tau)) * np.exp(- (a(t) + z)/(sigma*(t-tau)))
        term2 = (a(t)/z)**(0.5*(q-sigma)/sigma)
        term3 = da(t) - (a(t)/(t-tau)) + kk
        term4 = iv(q/sigma-1, 2*np.sqrt(a(t)*z)/(sigma*(t-tau)))
        term5 = (np.sqrt(a(t)*z)/(t-tau)) * iv(q/sigma, 2*np.sqrt(a(t)*z)/(sigma*(t-tau)))
    else:
        term1 = 1./(sigma*(t - tau))
        term2 = (a(t)/z)**(0.5*(q-sigma)/sigma)
        term3 = da(t) - (a(t)/(t-tau)) + kk
        term4 = ive(q/sigma-1, (a(t) + z)/(sigma*(t-tau)))
        term5 = (np.sqrt(a(t)*z)/(t-tau)) * ive(q/sigma, (a(t) + z)/(sigma*(t-tau)))
    
    return term1 * term2 * (term3 * term4 + term5)

def ie_bessel_fpt(a, da, q, z, sigma=2, dt=0.1, T_max=2):
    g = [0]
    T = [0]
    g.append(-2*psi(a, da, dt, z, 0, q, sigma))
    T.append(dt)
    
    for n in range(2, int(T_max/dt)+2):
        s = -2 * psi(a, da, n*dt, z, 0, q, sigma)

        for j in range(1, n):
            s += 2 * dt * g[j] * psi(a, da, n*dt, a(j*dt), j*dt, q, sigma)

        g.append(s)
        T.append(n*dt)
        
    g = np.asarray(g)
    T = np.asarray(T)
    
    gt = interp1d(T, g)
    return gt

In [4]:
def HSDM_4D_likelihood(prms, RT, Theta):
    a = lambda t: prms[0]**2
    da = lambda t: 0
    ndt = prms[1]
    
    eta = prms[2]
    eta2 = eta**2
    
    mu = np.array([prms[3], prms[4], prms[5], prms[6]])
    
    if max(RT) <= 4.5:
        fpt = ie_bessel_fpt(a, da, mu.shape[0], 0.000001, 
                            dt=0.02, T_max=max(RT))
    else:
        fpt = ie_bessel_fpt(a, da, mu.shape[0], 0.000001, 
                            dt=0.02, T_max=4.5)
        
        fpt_l = ie_bessel_fpt(a, da, mu.shape[0], 0.000001, 
                              dt=0.1, T_max=max(RT))
    
    log_lik = 0
    for i in range(len(RT)):
        rt, theta = RT[i], Theta[i]
        if rt - ndt > 0.001:
            x3 =  prms[0]*np.cos(theta[0])
            x2 =  prms[0]*np.sin(theta[0])*np.cos(theta[1])
            x1 =  prms[0]*np.sin(theta[0])*np.sin(theta[1])*np.cos(theta[2])
            x0 =  prms[0]*np.sin(theta[0])*np.sin(theta[1])*np.sin(theta[2])
            fixed = 1/(np.sqrt(eta2 * (rt - ndt) + 1))
            exponent0 = -0.5*mu[0]**2/eta2 + 0.5*(x0 * eta2 + mu[0])**2 / (eta2 * (eta2 * (rt - ndt) + 1))
            exponent1 = -0.5*mu[1]**2/eta2 + 0.5*(x1 * eta2 + mu[1])**2 / (eta2 * (eta2 * (rt - ndt) + 1))
            exponent2 = -0.5*mu[2]**2/eta2 + 0.5*(x2 * eta2 + mu[2])**2 / (eta2 * (eta2 * (rt - ndt) + 1))
            exponent3 = -0.5*mu[3]**2/eta2 + 0.5*(x3 * eta2 + mu[3])**2 / (eta2 * (eta2 * (rt - ndt) + 1))
            term1 = fixed * np.exp(exponent0)
            term2 = fixed * np.exp(exponent1)
            term3 = fixed * np.exp(exponent2)
            term4 = fixed * np.exp(exponent3)
            
            if rt - ndt <= 4.5:
                density = term1 * term2 * term3 * term4 * fpt(rt - ndt)
            else:
                density = term1 * term2 * term3 * term4 * fpt_l(rt - ndt)
            
            if 0.1**14 < density:
                log_lik += -np.log(density)
            else:
                log_lik += -np.log(0.1**14)
        else:
            log_lik += -np.log(0.1**14)
        
    return log_lik

In [5]:
min_threshold = 1
max_threshold = 3

min_ndt = 0.1
max_ndt = 1

min_eta = 0.1
max_eta = 1

min_mu = -2.5
max_mu = 2.5

file_name = '_Recovery_data/IE_4d_recovery_dvar.csv'

In [6]:
recovery_dic = {'threshold_true': [],
                'threshold_estimate': [],
                'ndt_true': [],
                'ndt_estimate': [],
                'eta_true': [],
                'eta_estimate': [],
                'mu1_true': [],
                'mu1_estimate': [],
                'mu2_true': [],
                'mu2_estimate': [],
                'mu3_true': [],
                'mu3_estimate': [],
                'mu4_true': [],
                'mu4_estimate': []}

In [7]:
for n in tqdm(range(200)):
    threshold = np.random.uniform(min_threshold, max_threshold)
    a = lambda t: threshold
    ndt = np.random.uniform(min_ndt, max_ndt)
    eta = np.random.uniform(min_eta, max_eta)
    mu = np.array([np.random.uniform(min_mu, max_mu), 
                   np.random.uniform(min_mu, max_mu),
                   np.random.uniform(min_mu, max_mu),
                   np.random.uniform(min_mu, max_mu)])
    
    RT = []
    Theta = []
    
    for i in range(500):
        rt, theta = simulate_HSDM_4D(a, mu, eta, ndt)
        RT.append(rt)
        Theta.append(theta)
    
    min_ans = differential_evolution(HSDM_4D_likelihood,
                                     args=(RT, Theta),
                                     bounds=[(min_threshold, max_threshold), 
                                             (min_ndt, max_ndt), (min_eta, max_eta),
                                             (min_mu, max_mu), (min_mu, max_mu), 
                                             (min_mu, max_mu), (min_mu, max_mu)])
    
    min_ans = minimize(HSDM_4D_likelihood,
                       method='nelder-mead',
                       x0=min_ans.x,
                       args=(RT, Theta),
                       bounds=[(min_threshold, max_threshold), 
                               (min_ndt, max_ndt), (min_eta, max_eta),
                               (min_mu, max_mu), (min_mu, max_mu), 
                               (min_mu, max_mu), (min_mu, max_mu)])
    
    if min_ans.success:
        recovery_dic['threshold_true'].append(threshold)
        recovery_dic['threshold_estimate'].append(min_ans.x[0])
        recovery_dic['ndt_true'].append(ndt)
        recovery_dic['ndt_estimate'].append(min_ans.x[1])
        recovery_dic['eta_true'].append(eta)
        recovery_dic['eta_estimate'].append(min_ans.x[2])
        recovery_dic['mu1_true'].append(mu[3])
        recovery_dic['mu1_estimate'].append(min_ans.x[3])
        recovery_dic['mu2_true'].append(mu[2])
        recovery_dic['mu2_estimate'].append(min_ans.x[4])
        recovery_dic['mu3_true'].append(mu[1])
        recovery_dic['mu3_estimate'].append(min_ans.x[5])
        recovery_dic['mu4_true'].append(mu[0])
        recovery_dic['mu4_estimate'].append(min_ans.x[6])
        
recovery_df = pd.DataFrame(recovery_dic)

100%|█████████████████████████████████████| 200/200 [10:43:49<00:00, 193.15s/it]


In [8]:
old_recovery_data = pd.read_csv(file_name, index_col=0)
recovery_df = pd.concat([old_recovery_data, recovery_df]).reset_index(drop=True)
recovery_df.to_csv(file_name)

In [9]:
recovery_df.shape[0]

250